In [56]:
import pandas as pd

# Convert CSV files into dataframes
orders = pd.read_csv('dataset/orders.csv')
order_products_prior = pd.read_csv('dataset/order_products__prior.csv')
products = pd.read_csv('dataset/products.csv')
aisles = pd.read_csv('dataset/aisles.csv')
departments = pd.read_csv('dataset/departments.csv')

In [57]:
def lightweight_eda(df):
    print(df.info())
    print("Number of missing values:")
    print(df.isna().sum())
    print("Number of duplicates:")
    print(df.duplicated().sum())


# Perform a lightweight EDA on each file
print("ORDERS")
print(orders['order_id'].value_counts())
lightweight_eda(orders)

print("PRODUCTS")
print(products['product_id'].value_counts())
lightweight_eda(products)

print("AISLES")
print(aisles['aisle_id'].value_counts())
lightweight_eda(aisles)

print("DEPARTMENTS")
print(departments['department_id'].value_counts())
lightweight_eda(departments)

print("ORDER_PRODUCTS")
lightweight_eda(order_products_prior)

ORDERS
order_id
2539329    1
1591157    1
1354759    1
1971373    1
1558866    1
          ..
3266950    1
118963     1
9433       1
2938641    1
272231     1
Name: count, Length: 3421083, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB
None
Number of missing values:
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64
Number of dupl

In [58]:
# Merge files into a single flat 
products_full = (products
    .merge(aisles, on='aisle_id', how='left')
    .merge(departments, on='department_id', how='left')
)

prior_products = order_products_prior.merge(products_full, on='product_id', how='left')

full_orders_products = prior_products.merge(
    orders,
    on='order_id',
    how='left'
)

# Convert object dtypes for efficient manipulation
full_orders_products['product_name'] = full_orders_products['product_name'].astype('string')
full_orders_products['department'] = full_orders_products['department'].astype('category')
full_orders_products['aisle'] = full_orders_products['aisle'].astype('category')


print(full_orders_products.head())

# Checking if there are instances where a product appears in an order multiple times
duplicates_within_orders = (
    full_orders_products_df
    .duplicated(subset=['order_id', 'product_name'], keep=False)
)
print("Number of (order_id, product) duplicates:",
      duplicates_within_orders.sum())

   order_id  product_id  add_to_cart_order  reordered           product_name  \
0         2       33120                  1          1     Organic Egg Whites   
1         2       28985                  2          1  Michigan Organic Kale   
2         2        9327                  3          0          Garlic Powder   
3         2       45918                  4          1         Coconut Butter   
4         2       30035                  5          0      Natural Sweetener   

   aisle_id  department_id               aisle  department  user_id eval_set  \
0        86             16                eggs  dairy eggs   202279    prior   
1        83              4    fresh vegetables     produce   202279    prior   
2       104             13   spices seasonings      pantry   202279    prior   
3        19             13       oils vinegars      pantry   202279    prior   
4        17             13  baking ingredients      pantry   202279    prior   

   order_number  order_dow  order_hour

In [59]:
# Remove columns that are not needed after merging
columns_to_remove = ["add_to_cart_order","reordered", "order_dow", "order_hour_of_day", "days_since_prior_order", "aisle_id", "department_id", "eval_set"]

truncated_df = full_orders_products.drop(columns=columns_to_remove)

truncated_df.head()

,order_id,product_id,product_name,aisle,department,user_id,order_number
0,2,33120,Organic Egg Whites,eggs,dairy eggs,202279,3
1,2,28985,Michigan Organic Kale,fresh vegetables,produce,202279,3
2,2,9327,Garlic Powder,spices seasonings,pantry,202279,3
3,2,45918,Coconut Butter,oils vinegars,pantry,202279,3
4,2,30035,Natural Sweetener,baking ingredients,pantry,202279,3


In [60]:
import seaborn as sns
import matplotlib.pyplot as plt


# Calculate order size
order_sizes = truncated_df.groupby('order_id')['product_id'].count()
order_sizes = order_sizes.reset_index(name='order_size')
order_size_counts = order_sizes['order_size'].value_counts().sort_index()

print(order_size_counts.head())

# Statistics for order size
print(order_sizes['order_size'].describe())

# Distribution for order sizes
plt.bar(order_size_counts.index, order_size_counts.values, color='skyblue', edgecolor='black')
plt.title("Order Size Distribution")
plt.xlabel("Number of Products per Order")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(f"order_size_frequency.png") 
plt.close()

# Get top 10 biggest orders
top_10_orders = order_sizes.sort_values(by='order_size', ascending=False).head(10)

print(top_10_orders)

order_size
1    156748
2    186993
3    207027
4    222081
5    228330
Name: count, dtype: int64
count    3.214874e+06
mean     1.008888e+01
std      7.525398e+00
min      1.000000e+00
25%      5.000000e+00
50%      8.000000e+00
75%      1.400000e+01
max      1.450000e+02
Name: order_size, dtype: float64
         order_id  order_size
1469463   1564244         145
742844     790903         137
57588       61355         127
2791358   2970392         121
1944784   2069920         116
3108595   3308010         115
2587216   2753324         114
2348898   2499774         112
2463426   2621625         109
72450       77151         109


In [61]:
# Calculate number of orders a product has appeared in
total_orders = truncated_df['order_id'].nunique()
product_counts = truncated_df['product_id'].value_counts().reset_index()
product_counts.columns = ['product_id', 'count']

# Add back the product names
product_counts = product_counts.merge(
    truncated_df[['product_id', 'product_name']].drop_duplicates(),
    on='product_id',
    how='left'
)
product_counts = product_counts[['product_id', 'product_name', 'count']]

# Add column to show percentage of orders containing each product
product_counts['order_penetration_pct'] = (product_counts['count'] / total_orders) * 100

print(product_counts.head())

plt.figure(figsize=(8, 5))
plt.hist(product_counts['count'], bins=50, color='skyblue', edgecolor='black')
plt.title("Distribution of Product Counts")
plt.xlabel("Number of Orders")
plt.ylabel("Number of Products")
plt.tight_layout()
plt.savefig(f"product_order_counts_hist.png") 
plt.close()

# Statistics for product counts
print(product_counts.describe())

# Sort ascending to get products in the fewest orders
top10_smallest = product_counts.sort_values('count').head(10)

print(top10_smallest)


   product_id            product_name   count  order_penetration_pct
0       24852                  Banana  472565              14.699332
1       13176  Bag of Organic Bananas  379450              11.802951
2       21137    Organic Strawberries  264683               8.233075
3       21903    Organic Baby Spinach  241921               7.525054
4       47209    Organic Hass Avocado  213584               6.643620
         product_id          count  order_penetration_pct
count  49677.000000   49677.000000           49677.000000
mean   24843.417356     652.907563               0.020309
std    14343.034804    4792.114416               0.149061
min        1.000000       1.000000               0.000031
25%    12423.000000      17.000000               0.000529
50%    24842.000000      60.000000               0.001866
75%    37264.000000     260.000000               0.008087
max    49688.000000  472565.000000              14.699332
       product_id                                product_name  c

In [62]:
# Count number of orders per user
user_order_counts = truncated_df.groupby('user_id')['order_id'].nunique()
user_order_counts = user_order_counts.reset_index(name='num_orders')

print(user_order_counts.head())

plt.hist(user_order_counts['num_orders'], bins=50, color='skyblue', edgecolor='black')
plt.xlabel("Number of Orders per User")
plt.ylabel("Number of Users")
plt.title("Distribution of Orders per User")
plt.savefig(f"user_order_counts_hist.png") 
plt.close()

# Statistics for product counts
print(user_order_counts.describe())

   user_id  num_orders
0        1          10
1        2          14
2        3          12
3        4           5
4        5           4
             user_id     num_orders
count  206209.000000  206209.000000
mean   103105.000000      15.590367
std     59527.555167      16.654774
min         1.000000       3.000000
25%     51553.000000       5.000000
50%    103105.000000       9.000000
75%    154657.000000      19.000000
max    206209.000000      99.000000


In [63]:
from ydata_profiling import ProfileReport


# Build a sample of users to keep order integrity and product diversity
sample_frac = 0.05
sampled_users = orders['user_id'].drop_duplicates().sample(frac=sample_frac, random_state=42)
df_sample = full_orders_products[full_orders_products['user_id'].isin(sampled_users)]

profile = ProfileReport(df_sample, explorative=True)
profile.to_file("eda_report_orders.html")

c:\Users\jenle\AppData\Local\Programs\Python\Python312\Lib\site-packages\ydata_profiling\utils\dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:10<00:00,  1.39it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]